In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,shutil
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist
from keras.callbacks import ReduceLROnPlateau
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras import layers
from keras import models
from keras import optimizers
from keras.applications import VGG19
from keras_radam import RAdam

Using TensorFlow backend.


In [2]:
from keras import backend as K


class Lookahead(object):
    """Add the [Lookahead Optimizer](https://arxiv.org/abs/1907.08610) functionality for [keras](https://keras.io/).
    """

    def __init__(self, k=5, alpha=0.5):
        self.k = k
        self.alpha = alpha
        self.count = 0

    def inject(self, model):
        """Inject the Lookahead algorithm for the given model.
        The following code is modified from keras's _make_train_function method.
        See: https://github.com/keras-team/keras/blob/master/keras/engine/training.py#L497
        """
        if not hasattr(model, 'train_function'):
            raise RuntimeError('You must compile your model before using it.')

        model._check_trainable_weights_consistency()

        if model.train_function is None:
            inputs = (model._feed_inputs +
                      model._feed_targets +
                      model._feed_sample_weights)
            if model._uses_dynamic_learning_phase():
                inputs += [K.learning_phase()]
            fast_params = model._collected_trainable_weights

            with K.name_scope('training'):
                with K.name_scope(model.optimizer.__class__.__name__):
                    training_updates = model.optimizer.get_updates(
                        params=fast_params,
                        loss=model.total_loss)
                    slow_params = [K.variable(p) for p in fast_params]
                fast_updates = (model.updates +
                                training_updates +
                                model.metrics_updates)

                slow_updates, copy_updates = [], []
                for p, q in zip(fast_params, slow_params):
                    slow_updates.append(K.update(q, q + self.alpha * (p - q)))
                    copy_updates.append(K.update(p, q))

                # Gets loss and metrics. Updates weights at each call.
                fast_train_function = K.function(
                    inputs,
                    [model.total_loss] + model.metrics_tensors,
                    updates=fast_updates,
                    name='fast_train_function',
                    **model._function_kwargs)

                def F(inputs):
                    self.count += 1
                    R = fast_train_function(inputs)
                    if self.count % self.k == 0:
                        K.batch_get_value(slow_updates)
                        K.batch_get_value(copy_updates)
                    return R
                
                model.train_function = F


In [3]:
LR_function=ReduceLROnPlateau(monitor='val_acc',
                             patience=3,
                             verbose=1,
                             factor=0.5,
                             min_lr=0.000001)



In [4]:
train_datagen = ImageDataGenerator(
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,                                   
                                   horizontal_flip=True,
                                   fill_mode='nearest')
                                   
train_batches = train_datagen.flow_from_directory('sample/train',
                                                  target_size=(224,224),
                                                  interpolation='bicubic',
                                                  class_mode='binary',
                                                  shuffle=True,
                                                  batch_size=64)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory('sample/valid',
                                                  target_size=(224,224),
                                                  interpolation='bicubic',
                                                  class_mode='binary',
                                                  shuffle=False,
                                                  batch_size=64)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
base_model=VGG19(input_shape=(224,224,3),include_top=False)
#base_model.trainable = False

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()
model.compile(optimizer=optimizers.SGD(lr=0.0001, momentum=0.9) , loss='binary_crossentropy' , metrics=['accuracy'])
lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
lookahead.inject(model) # add into model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 20,155,969
Trainable params: 20,155,969
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit_generator(train_batches,
                    steps_per_epoch = train_batches.samples // 64,
                    validation_data = valid_batches,
                    validation_steps = valid_batches.samples // 64,
                    epochs = 100,
                    callbacks=[LR_function])

Epoch 1/100
31/31 [==============================] - 87s 3s/step - loss: 1.0948 - acc: 0.6421 - val_loss: 0.3355 - val_acc: 0.8646
Epoch 2/100
31/31 [==============================] - 81s 3s/step - loss: 0.4073 - acc: 0.8130 - val_loss: 0.1943 - val_acc: 0.9220
Epoch 3/100
31/31 [==============================] - 76s 2s/step - loss: 0.3082 - acc: 0.8585 - val_loss: 0.1364 - val_acc: 0.9423
Epoch 4/100
31/31 [==============================] - 75s 2s/step - loss: 0.2217 - acc: 0.9087 - val_loss: 0.1159 - val_acc: 0.9487
Epoch 5/100
31/31 [==============================] - 76s 2s/step - loss: 0.1962 - acc: 0.9194 - val_loss: 0.0949 - val_acc: 0.9562
Epoch 6/100
31/31 [==============================] - 76s 2s/step - loss: 0.1769 - acc: 0.9309 - val_loss: 0.0813 - val_acc: 0.9679
Epoch 7/100
31/31 [==============================] - 76s 2s/step - loss: 0.1527 - acc: 0.9390 - val_loss: 0.0745 - val_acc: 0.9637
Epoch 8/100
31/31 [==============================] - 76s 2s/step - loss: 0.1326 - a

In [8]:
model.save('VGG19-3.h5')

In [7]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory('sample/test',
                                                  target_size=(224,224),
                                                  batch_size=64,
                                                  class_mode='binary')
model.evaluate_generator(test_generator, steps=50)
#test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
#print('test acc:', test_acc)

Found 1000 images belonging to 2 classes.


[0.07233505023409949, 0.9718670075201927]

In [9]:
import cv2
path = "./test/"

files=os.listdir(path)

X_test = []


for file in files:
        img_array = cv2.imread(os.path.join(path,file))
        new_img_array = cv2.resize(img_array, dsize=(224,224))
        X_test.append(new_img_array)   
        
X_test = np.array(X_test).reshape(-1,224,224,3)        


In [10]:
predictions = model.predict(X_test)

In [11]:
sub=pd.read_csv('sample_submission.csv')

for i in range(400):
    sub.loc[i,'Predicted']=predictions[i][0]
    
sub.to_csv("submission-9.csv", index=False)